- Exploration et analyse préliminaire des données
- Découpage des données en train/test
- Prétraitement en les convertissant en tenseurs
- Augmentation des données
- Définition des hyperparamètres
- Conception du modèle de CNN
- Compilation
- Entraînement
- Validation croisée
- Évaluation
- Affinement du modèle
- Test final

## Importation des bibliotheques

In [7]:
import os
import re
import random
import numpy as np
import shutil
from collections import defaultdict
from sklearn.model_selection import train_test_split
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import torch
import torch.nn as nn
import torchvision.transforms as transforms
import torchvision.datasets as datasets
from torch.utils.data import DataLoader
from timm import create_model
from sklearn.metrics import classification_report, confusion_matrix
import time 


## Affichage du nombre d'images par classes

In [8]:
# Chemin du dossier train_data
dossier_train_data = r"C:\Users\Christian\Desktop\UE Projet\Dataset\train_data"

# Dictionnaire pour stocker le nombre d'images par classe
nombre_images_par_classe = defaultdict(int)

# Parcourir tous les fichiers dans le dossier train_data
for file_name in os.listdir(dossier_train_data):
    """Les fichiers sont dans un dossiers train data déja prétraités. Nous allons extraire le nombre qui se trouve dans la structure du nom de chaque fichiers et conserver le nom de l'image qui represente la classe"""
    # Utiliser une expression régulière pour extraire la partie avant le nombre
    match = re.match(r"^(.*?)(\d+)\.jpeg$", file_name)
    if match:
        class_name = match.group(1)
        nombre_images_par_classe[class_name] += 1

# Afficher les résultats
print("Nombre d'images par classe dans train_data :")
for classe, nombre in nombre_images_par_classe.items():
    print(f"{classe} : {nombre}")


Nombre d'images par classe dans train_data :
Actiniaria_ : 37
Actinoptilum_molle_ : 12
Actinoscyphia_plebeia_ : 12
Actinostola_capensis_ : 11
Aequorea_spp_ : 24
Africolaria_rutila_ : 10
Alcyonacea_ : 13
Amalda_bullioides_ : 25
Anthoptilum_grandiflorum_ : 22
Aphelodoris_sp__ : 12
Aphrodita_alta_ : 12
Aristeus_varidens_ : 28
Armina_sp__ : 16
Ascidiacea_ : 14
Astropecten_irregularis_pontoporeus_ : 43
Athleta_abyssicola_ : 31
Athleta_lutosa_ : 28
Bolocera_kerguelensis_ : 28
Brissopsis_lyrifera_capensis_ : 32
Bryozoa_ : 17
Cavernularia_spp_ : 19
Cephalodiscus_gilchristi_ : 10
Ceramaster_patagonicus_euryplax_ : 12
Charonia_lampas_ : 13
Cheilostomatida_ : 12
Cheiraster_hirsutus_ : 25
Chondraster_elattosis_ : 10
Chrysaora_fulgida_ : 23
Chrysaora_spp_ : 16
Comanthus_wahlbergii_ : 16
Comitas_saldanhae_ : 45
Comitas_stolida_ : 11
Cosmasterias_felipes_ : 30
Crossaster_penicillatus_ : 24
Cypraeovula_iutsui_ : 17
Diplopteraster_multipes_ : 14
Dipsacaster_sladeni_capensis_ : 16
Echinus_gilchristi_ : 

## Decoupage en train/test (80/20)

In [9]:
# Chemin du dossier train_data qui contient les images d'entrainement
dossier_train_data = r"C:\Users\Christian\Desktop\UE Projet\Dataset\train_data"

# Definition des dossiers pour le train et le test
dossier_train = os.path.join(dossier_train_data, '..', 'train')
dossier_test = os.path.join(dossier_train_data, '..', 'test')


In [10]:
# Créer les dossiers train et test
os.makedirs(dossier_train, exist_ok=True)
os.makedirs(dossier_test, exist_ok=True)


In [11]:
"""Ici on va stocker les images par classes pour voir la classe a laquelle chaque image correspond"""
# Dictionnaire pour stocker les fichiers par classe
fichiers_par_classe = {}

# Parcourir tous les fichiers dans le dossier train_data
for file_name in os.listdir(dossier_train_data):
    # Utiliser une expression régulière pour extraire la classe
    match = re.match(r"^(.*?)(\d+)\.jpeg$", file_name)
    if match:
        class_name = match.group(1)
        if class_name not in fichiers_par_classe:
            fichiers_par_classe[class_name] = []
        fichiers_par_classe[class_name].append(file_name)
        

In [12]:
# Ici on affiche juste les images par classe.
fichiers_par_classe

{'Actiniaria_': ['Actiniaria_1.jpeg',
  'Actiniaria_10.jpeg',
  'Actiniaria_11.jpeg',
  'Actiniaria_12.jpeg',
  'Actiniaria_13.jpeg',
  'Actiniaria_14.jpeg',
  'Actiniaria_15.jpeg',
  'Actiniaria_16.jpeg',
  'Actiniaria_17.jpeg',
  'Actiniaria_18.jpeg',
  'Actiniaria_19.jpeg',
  'Actiniaria_2.jpeg',
  'Actiniaria_20.jpeg',
  'Actiniaria_21.jpeg',
  'Actiniaria_22.jpeg',
  'Actiniaria_23.jpeg',
  'Actiniaria_24.jpeg',
  'Actiniaria_25.jpeg',
  'Actiniaria_26.jpeg',
  'Actiniaria_27.jpeg',
  'Actiniaria_28.jpeg',
  'Actiniaria_29.jpeg',
  'Actiniaria_3.jpeg',
  'Actiniaria_30.jpeg',
  'Actiniaria_31.jpeg',
  'Actiniaria_32.jpeg',
  'Actiniaria_33.jpeg',
  'Actiniaria_34.jpeg',
  'Actiniaria_35.jpeg',
  'Actiniaria_36.jpeg',
  'Actiniaria_37.jpeg',
  'Actiniaria_4.jpeg',
  'Actiniaria_5.jpeg',
  'Actiniaria_6.jpeg',
  'Actiniaria_7.jpeg',
  'Actiniaria_8.jpeg',
  'Actiniaria_9.jpeg'],
 'Actinoptilum_molle_': ['Actinoptilum_molle_1.jpeg',
  'Actinoptilum_molle_10.jpeg',
  'Actinoptilum_mol

In [13]:
""" Extraction de 80% des images de chaque classes pour former l'ensemble d'entrainement et 20% pour l'ensemble de test"""
# Diviser les fichiers pour chaque classe
for class_name, fichiers in fichiers_par_classe.items():
    train_files, test_files = train_test_split(fichiers, test_size=0.2, random_state=42)
    
    # Copie des fichiers d'entraînement
    for file_name in train_files:
        src = os.path.join(dossier_train_data, file_name)
        dst = os.path.join(dossier_train, file_name)
        shutil.copy2(src, dst)
    
    # Copie des fichiers de test
    for file_name in test_files:
        src = os.path.join(dossier_train_data, file_name)
        dst = os.path.join(dossier_test, file_name)
        shutil.copy2(src, dst)

print("Les fichiers ont été divisés en ensembles d'entraînement et de test avec succès.")


Les fichiers ont été divisés en ensembles d'entraînement et de test avec succès.


### Les nouveaux chemins pour les ensembles d'entrainement et de test. Ici on rappel que tous les images de l'ensemble d'entrainement initial ont été decoupé en ensemble de train et de test pour concevoir un premier modele. Une fois qu'on aurait obtenu un model assez performant, on va l'entrainer sur l'ensemble de train total(train_small) et le tester sur le test(Test_small)

In [14]:
# Chemin du dossier train_data
dossier_train_data = r"C:\Users\Christian\Desktop\UE Projet\Dataset\train"

# Organiser les images en sous-dossiers
for file_name in os.listdir(dossier_train_data):
    # Utiliser une expression régulière pour extraire la classe
    match = re.match(r"^(.*?)(\d+)\.jpeg$", file_name)
    if match:
        class_name = match.group(1)
        class_dir = os.path.join(dossier_train_data, class_name)
        
        # Créer le sous-dossier si nécessaire
        if not os.path.exists(class_dir):
            os.makedirs(class_dir)
        
        # Déplacer l'image dans le sous-dossier
        src_path = os.path.join(dossier_train_data, file_name)
        dst_path = os.path.join(class_dir, file_name)
        shutil.move(src_path, dst_path)

print("Images réorganisées en sous-dossiers par classe.")


Images réorganisées en sous-dossiers par classe.


In [15]:
# Chemin du dossier test_data
dossier_test_data = r"C:\Users\Christian\Desktop\UE Projet\Dataset\test"

# Organiser les images en sous-dossiers
for file_name in os.listdir(dossier_test_data):
    # Utiliser une expression régulière pour extraire la classe
    match = re.match(r"^(.*?)(\d+)\.jpeg$", file_name)
    if match:
        class_name = match.group(1)
        class_dir = os.path.join(dossier_test_data, class_name)
        
        # Créer le sous-dossier si nécessaire
        if not os.path.exists(class_dir):
            os.makedirs(class_dir)
        
        # Déplacer l'image dans le sous-dossier
        src_path = os.path.join(dossier_test_data, file_name)
        dst_path = os.path.join(class_dir, file_name)
        shutil.move(src_path, dst_path)

print("Images de test réorganisées en sous-dossiers par classe.")


Images de test réorganisées en sous-dossiers par classe.


## Creation d'un dataset a partir des données d'entrainement et de test pour le passage dans le modele et augmentation de la données

In [16]:
# Les données d'entrainement et de test
dossier_train = r"C:\Users\Christian\Desktop\UE Projet\Dataset\train"
dossier_test = r"C:\Users\Christian\Desktop\UE Projet\Dataset\test"

# Définitions des paramètres pour la taille des images et le nombres d'images traités par lot
IMG_SIZE = (128, 128)
BATCH_SIZE = 32

# Créer un dataset à partir des fichiers d'entraînement
train_dataset = tf.keras.preprocessing.image_dataset_from_directory(
    directory=dossier_train,
    labels='inferred', # Pour specifier que chaque images de chaque sous repertoires appartient uniquement a cette classe
    label_mode='int', # Encodage en entier pour convertir les labels(classes/sous repertoire) en entier afin de faciliter l'entrainement
    batch_size=BATCH_SIZE,
    image_size=IMG_SIZE,
    shuffle=True
)

# Créer un dataset à partir des fichiers de test
test_dataset = tf.keras.preprocessing.image_dataset_from_directory(
    directory=dossier_test,
    labels='inferred',
    label_mode='int',
    batch_size=BATCH_SIZE,
    image_size=IMG_SIZE,
    shuffle=False
)

# Normaliser les images
normalization_layer = tf.keras.layers.Rescaling(1./255)

# Appliquer la normalisation aux datasets
train_dataset = train_dataset.map(lambda x, y: (normalization_layer(x), y))
test_dataset = test_dataset.map(lambda x, y: (normalization_layer(x), y))

# Afficher la structure des datasets
for images, labels in train_dataset.take(1):
    print(images.shape)  # Afficher la forme des tenseurs d'images
    print(labels.shape)  # Afficher la forme des tenseurs de labels


Found 2437 files belonging to 137 classes.
Found 674 files belonging to 137 classes.
(32, 128, 128, 3)
(32,)


### Application de la data augmentation

Grace a cette technique, lors de l'entrainement du modele sur 10 epoques, le modele aura un equivalent de 24370 images dans sa base d'entrainement. Le principe est le suivant: Lors de l'entrainement du modele sur chaque époques, chaque image recevra les transformations suivante avant d'etre reconnu par le modele comme element d'une classe.

In [19]:
# Chemin des dossiers train et test
dossier_train = r"C:\Users\Christian\Desktop\UE Projet\Dataset\train"
dossier_test = r"C:\Users\Christian\Desktop\UE Projet\Dataset\test"

# Créer un générateur d'images avec augmentation pour l'entraînement
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

# Appliquer le générateur aux données d'entraînement
train_generator = train_datagen.flow_from_directory(
    directory=dossier_train,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='sparse'  # Utiliser 'sparse' pour les étiquettes entières
)

"""Nous allons appliquer cette section dans l'entrainement du modele final en considerant l'ensemble de test de train_data comme ensemble de validation"""
# Créer un générateur de données de validation sans augmentation, juste rescalage
"""test_datagen = ImageDataGenerator(rescale=1./255)
test_generator = test_datagen.flow_from_directory(
    directory=dossier_test,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='sparse',
    shuffle=False)"""


# Afficher la structure des datasets
for images, labels in train_generator:
    print(images.shape)  # Afficher la forme des tenseurs d'images
    print(labels.shape)  # Afficher la forme des tenseurs de labels
    break  # Pour éviter d'afficher toutes les images


Found 2437 images belonging to 137 classes.
(32, 128, 128, 3)
(32,)


### Vérifieons le contenu du répertoire train avant son passage dans le modele et voyons a quoi correspond chaque classe de l'ensemble d'entrainement par rapport aux classes de depart. On va verifier cela avec les indice car plus haut nous avons utiliser un encodage pour encoder nos differentes classes en valeurs entieres (le target encoding) pour faciliter le passage dans le modele


In [20]:
# Chemin du dossier d'entrainement
dossier_train = r"C:\Users\Christian\Desktop\UE Projet\Dataset\train"
# Affichage des classes
print("Contenu du répertoire 'train' :")
for file_name in os.listdir(dossier_train):
    print(file_name)


Contenu du répertoire 'train' :
Actiniaria_
Actinoptilum_molle_
Actinoscyphia_plebeia_
Actinostola_capensis_
Aequorea_spp_
Africolaria_rutila_
Alcyonacea_
Amalda_bullioides_
Anthoptilum_grandiflorum_
Aphelodoris_sp__
Aphrodita_alta_
Aristeus_varidens_
Armina_sp__
Ascidiacea_
Astropecten_irregularis_pontoporeus_
Athleta_abyssicola_
Athleta_lutosa_
Bolocera_kerguelensis_
Brissopsis_lyrifera_capensis_
Bryozoa_
Cavernularia_spp_
Cephalodiscus_gilchristi_
Ceramaster_patagonicus_euryplax_
Charonia_lampas_
Cheilostomatida_
Cheiraster_hirsutus_
Chondraster_elattosis_
Chrysaora_fulgida_
Chrysaora_spp_
Comanthus_wahlbergii_
Comitas_saldanhae_
Comitas_stolida_
Cosmasterias_felipes_
Crossaster_penicillatus_
Cypraeovula_iutsui_
Diplopteraster_multipes_
Dipsacaster_sladeni_capensis_
Echinus_gilchristi_
Eleutherobia_variable_
Euspira_napus_
Exodromidia_spinosa_
Exodromidia_spinosissima_
Flabellum_(Ulocyathus)_messum_
Funchalia_woodwardi_
Fusinus_africanae_
Fusinus_hayesi_
Fusitriton_magellanicus_
Fus

In [21]:
# Récupérer les noms des classes
class_names = train_generator.class_indices
class_names = {v: k for k, v in class_names.items()}  # Inverser le dictionnaire pour obtenir les noms des classes

# Affichage des noms des classes 
print("Classes et indices correspondants :")
for idx, class_name in class_names.items():
    print(f"Indice {idx} : {class_name}")


Classes et indices correspondants :
Indice 0 : Actiniaria_
Indice 1 : Actinoptilum_molle_
Indice 2 : Actinoscyphia_plebeia_
Indice 3 : Actinostola_capensis_
Indice 4 : Aequorea_spp_
Indice 5 : Africolaria_rutila_
Indice 6 : Alcyonacea_
Indice 7 : Amalda_bullioides_
Indice 8 : Anthoptilum_grandiflorum_
Indice 9 : Aphelodoris_sp__
Indice 10 : Aphrodita_alta_
Indice 11 : Aristeus_varidens_
Indice 12 : Armina_sp__
Indice 13 : Ascidiacea_
Indice 14 : Astropecten_irregularis_pontoporeus_
Indice 15 : Athleta_abyssicola_
Indice 16 : Athleta_lutosa_
Indice 17 : Bolocera_kerguelensis_
Indice 18 : Brissopsis_lyrifera_capensis_
Indice 19 : Bryozoa_
Indice 20 : Cavernularia_spp_
Indice 21 : Cephalodiscus_gilchristi_
Indice 22 : Ceramaster_patagonicus_euryplax_
Indice 23 : Charonia_lampas_
Indice 24 : Cheilostomatida_
Indice 25 : Cheiraster_hirsutus_
Indice 26 : Chondraster_elattosis_
Indice 27 : Chrysaora_fulgida_
Indice 28 : Chrysaora_spp_
Indice 29 : Comanthus_wahlbergii_
Indice 30 : Comitas_sald

## Utilisations d'un ViTf (Vision Transformers) pré-entrainé:

In [22]:
# Chemin du dataset
data_dir = r'C:\Users\Christian\Desktop\UE Projet\Dataset\train'

# Préparer les transformations et le DataLoader
transform = transforms.Compose([
    transforms.Resize((224, 224)), # Redimensionnement des images. generalemet les données entraineées sur modele utilise cette structure. alors j'ai preféré la conserver plutot que que de garder notre taille initial de (128,128)
    # transforms.RandomRotation(degrees=40),
    # transforms.RandomHorizontalFlip(p=0.5),
    # transforms.RandomResizedCrop(size=224, scale=(0.8, 1.0)),
    # transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.2),
    transforms.ToTensor(), # Ici on normalise les images en les fesant passer de la plage [0,255] en la plage [0,1] car initialement les images sont des tenseurs numpy
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]), # Normalisation appliqué au format des couleurs RGB pour une moyenne de mean et un ecart type de std
]) # On aurait pu ajouter d'autre transformation mais ca serait une redondance car cela a deja été fait dans notre section data augmentation.

# Application de ce second prétraitement aux données de la base train
train_dataset = datasets.ImageFolder(root=data_dir, transform=transform)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

# Charger le modèle ViT pré-entraîné depuis timm avec 137 sorties correspondant au 137 classes
model = create_model('vit_base_patch16_224', pretrained=True)
model.head = nn.Linear(model.head.in_features, 137)

# Définir l'optimiseur et la fonction de perte
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)


model.safetensors:   0%|          | 0.00/346M [00:00<?, ?B/s]

Error while downloading from https://cdn-lfs.hf.co/repos/fb/cd/fbcdc88e492959e3ee2515f497fb45cb5f217f9455c204bdf4e0b400c90d0c23/32aa17d6e17b43500f531d5f6dc9bc93e56ed8841b8a75682e1bb295d722405b?response-content-disposition=inline%3B+filename*%3DUTF-8%27%27model.safetensors%3B+filename%3D%22model.safetensors%22%3B&Expires=1736688222&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTczNjY4ODIyMn19LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy5oZi5jby9yZXBvcy9mYi9jZC9mYmNkYzg4ZTQ5Mjk1OWUzZWUyNTE1ZjQ5N2ZiNDVjYjVmMjE3Zjk0NTVjMjA0YmRmNGUwYjQwMGM5MGQwYzIzLzMyYWExN2Q2ZTE3YjQzNTAwZjUzMWQ1ZjZkYzliYzkzZTU2ZWQ4ODQxYjhhNzU2ODJlMWJiMjk1ZDcyMjQwNWI%7EcmVzcG9uc2UtY29udGVudC1kaXNwb3NpdGlvbj0qIn1dfQ__&Signature=jq2YeEH48ZyNOVchXzrJHTRQf1IyYVHWpe7tZTFAZbzqAQ0wzWekNTdkwJoXeUZQiSVU5sch0CPQ3I6mchsVc4VzvaHVF%7EUA2R0iPujevtUhnldpuihU0mPsmQtXIpUHeRijigGo6JkMnLVxZw8p9mEsJFCt0bt719I784vnqQkEz6EIt8mFlYLkNTcLWrpygtY%7EbkejOvaGg6U7EgshCjGGLCnku5i%7E7F4uTukKDRzf1dn5nEmRrWklyCqDA1LQB3wOO

model.safetensors:   3%|3         | 10.5M/346M [00:00<?, ?B/s]

Error while downloading from https://cdn-lfs.hf.co/repos/fb/cd/fbcdc88e492959e3ee2515f497fb45cb5f217f9455c204bdf4e0b400c90d0c23/32aa17d6e17b43500f531d5f6dc9bc93e56ed8841b8a75682e1bb295d722405b?response-content-disposition=inline%3B+filename*%3DUTF-8%27%27model.safetensors%3B+filename%3D%22model.safetensors%22%3B&Expires=1736688222&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTczNjY4ODIyMn19LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy5oZi5jby9yZXBvcy9mYi9jZC9mYmNkYzg4ZTQ5Mjk1OWUzZWUyNTE1ZjQ5N2ZiNDVjYjVmMjE3Zjk0NTVjMjA0YmRmNGUwYjQwMGM5MGQwYzIzLzMyYWExN2Q2ZTE3YjQzNTAwZjUzMWQ1ZjZkYzliYzkzZTU2ZWQ4ODQxYjhhNzU2ODJlMWJiMjk1ZDcyMjQwNWI%7EcmVzcG9uc2UtY29udGVudC1kaXNwb3NpdGlvbj0qIn1dfQ__&Signature=jq2YeEH48ZyNOVchXzrJHTRQf1IyYVHWpe7tZTFAZbzqAQ0wzWekNTdkwJoXeUZQiSVU5sch0CPQ3I6mchsVc4VzvaHVF%7EUA2R0iPujevtUhnldpuihU0mPsmQtXIpUHeRijigGo6JkMnLVxZw8p9mEsJFCt0bt719I784vnqQkEz6EIt8mFlYLkNTcLWrpygtY%7EbkejOvaGg6U7EgshCjGGLCnku5i%7E7F4uTukKDRzf1dn5nEmRrWklyCqDA1LQB3wOO

model.safetensors:   9%|9         | 31.5M/346M [00:00<?, ?B/s]

C:\Users\Christian\AppData\Roaming\Python\Python312\site-packages\huggingface_hub\file_download.py:140: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Christian\.cache\huggingface\hub\models--timm--vit_base_patch16_224.augreg2_in21k_ft_in1k. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


### Entrainement du model sur 10 epoques

In [23]:
# Entrainement du model avec affichage de la précision et de la perte
for epoch in range(10):  
    start_time = time.time() # Début de l'époque
    correct_train = 0 # Nombre total d'image correctement predit dans cette epoque
    total_train = 0 # Nombre total d'image vu dans cette epoque
    for images, labels in train_loader:
        optimizer.zero_grad() # Reinitialisation des gradients
        outputs = model(images) # Predictions
        loss = criterion(outputs, labels) # Loss
        loss.backward() # Retropropagation
        optimizer.step() # Ajustement des parametres
        
        _, predicted = torch.max(outputs.data, 1) # On reccupere l'indice de la classe predite avec la plus grande probabilité
        total_train += labels.size(0) # Mise a jour du nombre d'image vu dans cette epoque jusqu'a cette etapes
        correct_train += (predicted == labels).sum().item() # Mise a jour du Nombre total d'image correctement predit dans cette epoque
    
    train_accuracy = 100 * correct_train / total_train # Calcul de la precision apres chaque etapes
    """end_time = time.time() # Fin de l'époque 
    epoch_time = end_time - start_time # Calcul du temps d'exécution 
    print(f"Epoch [{epoch+1}/10], Loss: {loss.item():.4f}, Train Accuracy: {train_accuracy:.2f}%, Time: {epoch_time:.2f} seconds")"""
    print(f"Epoch [{epoch+1}/10], Loss: {loss.item():.4f}, Train Accuracy: {train_accuracy:.2f}%") # affichage de l'epoque, de la perte et de la precision


Epoch [1/10], Loss: 0.7068, Train Accuracy: 34.88%
Epoch [2/10], Loss: 0.2448, Train Accuracy: 82.31%
Epoch [3/10], Loss: 0.0212, Train Accuracy: 95.53%
Epoch [4/10], Loss: 0.0245, Train Accuracy: 99.59%
Epoch [5/10], Loss: 0.0002, Train Accuracy: 100.00%
Epoch [6/10], Loss: 0.0006, Train Accuracy: 100.00%
Epoch [7/10], Loss: 0.0002, Train Accuracy: 100.00%
Epoch [8/10], Loss: 0.0003, Train Accuracy: 100.00%
Epoch [9/10], Loss: 0.0004, Train Accuracy: 100.00%
Epoch [10/10], Loss: 0.0002, Train Accuracy: 100.00%


### Evaluation du modele sur l'ensemble de test

In [24]:
# Préparer les données de test
test_data_dir = r'C:\Users\Christian\Desktop\UE Projet\Dataset\test' 
test_dataset = datasets.ImageFolder(root=test_data_dir, transform=transform)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# Évaluation du modèle
model.eval()
correct_test = 0
total_test = 0
all_preds = []
all_labels = []

with torch.no_grad():
    for images, labels in test_loader:
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total_test += labels.size(0)
        correct_test += (predicted == labels).sum().item()
        
        all_preds.append(predicted.cpu().numpy())
        all_labels.append(labels.cpu().numpy())

test_accuracy = 100 * correct_test / total_test
print(f'Test Accuracy: {test_accuracy:.2f}%')


Test Accuracy: 88.58%


In [25]:
# Générer le rapport de classification
y_pred_classes = np.concatenate(all_preds)
y_true = np.concatenate(all_labels)
class_names = [str(i) for i in range(137)] 
report = classification_report(y_true, y_pred_classes, target_names=class_names, zero_division=0)
print(report)

              precision    recall  f1-score   support

           0       0.67      1.00      0.80         8
           1       0.50      0.67      0.57         3
           2       1.00      1.00      1.00         3
           3       1.00      0.33      0.50         3
           4       1.00      1.00      1.00         5
           5       1.00      0.50      0.67         2
           6       1.00      1.00      1.00         3
           7       1.00      1.00      1.00         5
           8       1.00      0.80      0.89         5
           9       0.75      1.00      0.86         3
          10       1.00      1.00      1.00         3
          11       0.62      0.83      0.71         6
          12       1.00      1.00      1.00         4
          13       1.00      0.33      0.50         3
          14       1.00      1.00      1.00         9
          15       0.88      1.00      0.93         7
          16       1.00      0.83      0.91         6
          17       0.83    

### Analyse du rapport de classification
#### les classes avec une precision inferieur ou egale a 25% 

In [26]:
# Récupérer les noms des classes depuis le train_generator
class_names = train_generator.class_indices
class_names = {v: k for k, v in class_names.items()}  # Inverser le dictionnaire pour obtenir les noms des classes

# Générer le rapport de classification sous forme de dictionnaire
report = classification_report(y_true, y_pred_classes, target_names=[class_names[i] for i in range(len(class_names))], output_dict=True, zero_division=0)

# Extraire les classes ayant une précision ≤ 25%
low_precision_classes = {class_name: metrics for class_name, metrics in report.items() if isinstance(metrics, dict) and 'precision' in metrics and metrics['precision'] <= 0.25}

# Afficher les classes et leurs précisions, rappels, supports
print("Classes avec précision ≤ 25%:")
for class_name, metrics in low_precision_classes.items():
    precision = metrics['precision']
    recall = metrics['recall']
    support = metrics['support']
    print(f"Classe: {class_name}, Précision: {precision:.2f}, Rappel: {recall:.2f}, Support: {int(support)}")


Classes avec précision ≤ 25%:
Classe: Polychaete_tubes_(only)_, Précision: 0.00, Rappel: 0.00, Support: 3


#### Extraction des classes ayant une précision > 25% et <= 50%

In [27]:
medium_precision_classes = {
    class_name: metrics for class_name, metrics in report.items()
    if isinstance(metrics, dict) and 'precision' in metrics and metrics['precision'] > 0.25 and metrics['precision'] <= 0.50
}

# Afficher les classes et leurs précisions, rappels, supports
print("Classes avec précision > 25% et inferieur a 50%:")
for class_name, metrics in medium_precision_classes.items():
    precision = metrics['precision']
    recall = metrics['recall']
    support = metrics['support']
    print(f"Classe: {class_name}, Précision: {precision:.2f}, Rappel: {recall:.2f}, Support: {int(support)}")


Classes avec précision > 25% et inferieur a 50%:
Classe: Actinoptilum_molle_, Précision: 0.50, Rappel: 0.67, Support: 3
Classe: Chondraster_elattosis_, Précision: 0.50, Rappel: 1.00, Support: 2
Classe: Diplopteraster_multipes_, Précision: 0.50, Rappel: 0.33, Support: 3
Classe: Philine_aperta_, Précision: 0.50, Rappel: 0.33, Support: 3


#### Erreur de classification

In [28]:
# Calculer la matrice de confusion
conf_matrix = confusion_matrix(y_true, y_pred_classes)

# Afficher les erreurs de classification
print("Erreur de classification")
for i, (true_class, pred_class) in enumerate(zip(y_true, y_pred_classes)):
    if true_class != pred_class:
        # Afficher les informations sur l'erreur
        print(f'Image {i}: Vraie classe = {class_names[true_class]}, Prédite = {class_names[pred_class]}')
        

Erreur de classification
Image 10: Vraie classe = Actinoptilum_molle_, Prédite = Hemiocnus_insolens_
Image 14: Vraie classe = Actinostola_capensis_, Prédite = Chondraster_elattosis_
Image 16: Vraie classe = Actinostola_capensis_, Prédite = Chondraster_elattosis_
Image 22: Vraie classe = Africolaria_rutila_, Prédite = Neptuneopsis_gilchristi_
Image 32: Vraie classe = Anthoptilum_grandiflorum_, Prédite = Bolocera_kerguelensis_
Image 48: Vraie classe = Aristeus_varidens_, Prédite = Prawns_
Image 53: Vraie classe = Ascidiacea_, Prédite = Porifera_
Image 54: Vraie classe = Ascidiacea_, Prédite = Pleurobranchaea_bubala_
Image 76: Vraie classe = Athleta_lutosa_, Prédite = Athleta_abyssicola_
Image 79: Vraie classe = Bolocera_kerguelensis_, Prédite = Lamellaria_Coriocella_spp_
Image 98: Vraie classe = Cavernularia_spp_, Prédite = Kaloplocamus_ramosus_
Image 108: Vraie classe = Cheilostomatida_, Prédite = Hydrozoa_spp_
Image 144: Vraie classe = Cosmasterias_felipes_, Prédite = Perissasterias_po

#### Classification correcte

In [29]:
# Affichage des images bien classés.
for i, (true_class, pred_class) in enumerate(zip(y_true, y_pred_classes)):
    if true_class == pred_class:
        print(f'Image {i}: Vraie classe = {class_names[true_class]}, Prédite = {class_names[pred_class]}')


Image 0: Vraie classe = Actiniaria_, Prédite = Actiniaria_
Image 1: Vraie classe = Actiniaria_, Prédite = Actiniaria_
Image 2: Vraie classe = Actiniaria_, Prédite = Actiniaria_
Image 3: Vraie classe = Actiniaria_, Prédite = Actiniaria_
Image 4: Vraie classe = Actiniaria_, Prédite = Actiniaria_
Image 5: Vraie classe = Actiniaria_, Prédite = Actiniaria_
Image 6: Vraie classe = Actiniaria_, Prédite = Actiniaria_
Image 7: Vraie classe = Actiniaria_, Prédite = Actiniaria_
Image 8: Vraie classe = Actinoptilum_molle_, Prédite = Actinoptilum_molle_
Image 9: Vraie classe = Actinoptilum_molle_, Prédite = Actinoptilum_molle_
Image 11: Vraie classe = Actinoscyphia_plebeia_, Prédite = Actinoscyphia_plebeia_
Image 12: Vraie classe = Actinoscyphia_plebeia_, Prédite = Actinoscyphia_plebeia_
Image 13: Vraie classe = Actinoscyphia_plebeia_, Prédite = Actinoscyphia_plebeia_
Image 15: Vraie classe = Actinostola_capensis_, Prédite = Actinostola_capensis_
Image 17: Vraie classe = Aequorea_spp_, Prédite = Ae

In [30]:
# Chemin complet où vous souhaitez sauvegarder le modèle
save_path = r'C:\Users\Christian\Desktop\UE Projet\model_vit.pth'

# Sauvegarder le modèle
torch.save(model.state_dict(), save_path)
print(f"Modèle sauvegardé à l'emplacement : {save_path}")


Modèle sauvegardé à l'emplacement : C:\Users\Christian\Desktop\UE Projet\model_vit.pth


In [31]:
import torch

# Chemin complet où vous souhaitez sauvegarder le modèle
save_path = r'C:\Users\Christian\Desktop\UE Projet\model_vit.pth'

# Sauvegarder le modèle complet
torch.save(model, save_path)
